In [1]:
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
import driving_data
import model
import numpy as np
import time
import config as cfg

begin = time.strftime('%Y-%m-%d_%H-%M-%S')

LOGDIR = './save'

tf.logging.set_verbosity(tf.logging.ERROR)

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

start_learning_rate = 0.5e-3  # 1e-3
adjust_learning_rate = 1e-5

#onehot_labels = tf.one_hot(indices=tf.reshape(tf.cast(model.y_, tf.int32),[-1]), depth=4)
onehot_labels = tf.one_hot(indices=tf.reshape(tf.cast(model.y_, tf.int32),[-1]), depth=cfg.NUM_KEYS)

loss = tf.losses.softmax_cross_entropy( onehot_labels=onehot_labels, logits=model.y)
# loss = tf.nn.softmax_cross_entropy_with_logits_v2( labels=onehot_labels, logits=model.y)
train_step = tf.train.AdamOptimizer(start_learning_rate).minimize(loss)

loss_val = tf.losses.softmax_cross_entropy( onehot_labels=onehot_labels, logits=model.y)
# loss_val = tf.nn.softmax_cross_entropy_with_logits_v2( labels=onehot_labels, logits=model.y)
# train_step = tf.train.AdamOptimizer(start_learning_rate).minimize(loss)

# sess.run(tf.initialize_all_variables())
sess.run(tf.global_variables_initializer())

# create a summary to monitor cost tensor
# tf.scalar_summary("loss", loss)
tf.summary.scalar("loss", loss)

tf.summary.scalar("loss_val", loss_val)

# merge all summaries into a single op
# merged_summary_op = tf.merge_all_summaries()
merged_summary_op = tf.summary.merge_all()

saver = tf.train.Saver(write_version = tf.train.SaverDef.V2)
# saver.restore(sess, "save/model.ckpt")

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

#epochs = 30
epochs = 30
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
    for i in range(int(driving_data.num_images/batch_size)):
        xs, ys = driving_data.LoadTrainBatch(batch_size)
        # train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.8})
        train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.7})
        loss_value = loss.eval(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 1.0})
        # print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))
        print("Epoch: %d, Step: %d, Loss: %g" % (epoch, i, loss_value))

        if i % 10 == 0:
            xs_val, ys_val = driving_data.LoadValBatch(batch_size)
            # xs, ys = driving_data.LoadTrainBatch(batch_size)
            loss_val = loss.eval(feed_dict={model.x:xs_val, model.y_: ys_val, model.keep_prob: 1.0})
            print("Epoch: %d, Step: %d, Loss_val: %g" % (epoch, i, loss_val))

        # write logs at every iteration
        summary = merged_summary_op.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
        summary_writer.add_summary(summary, epoch * driving_data.num_images/batch_size + i)

        if i % batch_size == 0:
            if not os.path.exists(LOGDIR):
                os.makedirs(LOGDIR)
            checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
            filename = saver.save(sess, checkpoint_path)
            print("Model saved in file: %s" % filename)


correct_prediction = tf.equal(tf.argmax(onehot_labels, 1), tf.argmax(model.y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print('Train Accuracy:', sess.run(accuracy, feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 1.0}))
print('Validation Accuracy:', sess.run(accuracy, feed_dict={model.x: xs_val, model.y_: ys_val, model.keep_prob: 1.0}))

end = time.strftime('%Y-%m-%d_%H-%M-%S')
print('begin: ', begin)
print('end: ', end)

print("Run the command line:\n",
      "--> tensorboard --logdir=./logs --port=6006",
      "\nThen open http://0.0.0.0:6006/ into your web browser")

# os.system("sudo -s shutdown -h now")




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
model read
Epoch: 0, Step: 0, Loss: 1.80965
Epoch: 0, Step: 0, Loss_val: 1.79637
Model saved in file: ./save/model.ckpt
Epoch: 0, Step: 1, Loss: 1.80599
Epoch: 0, Step: 2, Loss: 1.78807
Epoch: 0, Step: 3, Loss: 1.81284
Epoch: 0, Step: 4, Loss: 1.79099
Epoch: 0, Step: 5, Loss: 1.79075
Epoch: 0, Step: 6, Loss: 1.78893
Epoch: 0, Step: 7, Loss: 1.79167
Epoch: 0, Step: 8, Loss: 1.79114
Epoch: 0, Step: 9, Loss: 1.79127
Epoch: 0, Step: 10, Loss: 1.7914
Epoch: 0, Step: 10, Loss_val: 1.79162
Epoch: 0, Step: 11, Loss: 1.7913
Epoch: 0, Step: 12, Loss: 1.79179
Epoch: 0, Step: 13, Loss: 1.79207
Epoch: 0, Step: 14, Loss: 1.79046
Epoch: 0, Step: 15, Loss: 1.79085
Epoch: 0, Step: 16, Loss: 1.79059
Epoch: 0, Step: 17, Loss: 1.78995
Epoch: 0, Step: 18, Loss: 1.79196
Epoch: 0, Step: 19, Loss: 1.79159
Epoch: 0, Step: 20, Loss: 1.79025
Epoch: 0, Step: 20, Loss_val: 1.79089
Epoch: 0, Step: 21

Epoch: 0, Step: 193, Loss: 0.768911
Epoch: 0, Step: 194, Loss: 0.59519
Epoch: 0, Step: 195, Loss: 0.522665
Epoch: 0, Step: 196, Loss: 0.755187
Epoch: 0, Step: 197, Loss: 0.585849
Epoch: 0, Step: 198, Loss: 0.806909
Epoch: 0, Step: 199, Loss: 0.715124
Epoch: 0, Step: 200, Loss: 0.577211
Epoch: 0, Step: 200, Loss_val: 0.529992
Model saved in file: ./save/model.ckpt
Epoch: 0, Step: 201, Loss: 0.576083
Epoch: 0, Step: 202, Loss: 0.637198
Epoch: 0, Step: 203, Loss: 0.610258
Epoch: 0, Step: 204, Loss: 0.817952
Epoch: 0, Step: 205, Loss: 0.664126
Epoch: 0, Step: 206, Loss: 0.759299
Epoch: 0, Step: 207, Loss: 0.610543
Epoch: 0, Step: 208, Loss: 0.744517
Epoch: 0, Step: 209, Loss: 0.568609
Epoch: 0, Step: 210, Loss: 0.741278
Epoch: 0, Step: 210, Loss_val: 0.658394
Epoch: 0, Step: 211, Loss: 0.636628
Epoch: 0, Step: 212, Loss: 0.6113
Epoch: 0, Step: 213, Loss: 0.679837
Epoch: 0, Step: 214, Loss: 0.76155
Epoch: 0, Step: 215, Loss: 0.633772
Epoch: 0, Step: 216, Loss: 0.606499
Epoch: 0, Step: 217, 

Epoch: 0, Step: 399, Loss: 0.404145
Epoch: 0, Step: 400, Loss: 0.384877
Epoch: 0, Step: 400, Loss_val: 0.325238
Model saved in file: ./save/model.ckpt
Epoch: 0, Step: 401, Loss: 0.480334
Epoch: 1, Step: 0, Loss: 0.393384
Epoch: 1, Step: 0, Loss_val: 0.473161
Model saved in file: ./save/model.ckpt
Epoch: 1, Step: 1, Loss: 0.395264
Epoch: 1, Step: 2, Loss: 0.466231
Epoch: 1, Step: 3, Loss: 0.318201
Epoch: 1, Step: 4, Loss: 0.390444
Epoch: 1, Step: 5, Loss: 0.300173
Epoch: 1, Step: 6, Loss: 0.401995
Epoch: 1, Step: 7, Loss: 0.492345
Epoch: 1, Step: 8, Loss: 0.329686
Epoch: 1, Step: 9, Loss: 0.356031
Epoch: 1, Step: 10, Loss: 0.475598
Epoch: 1, Step: 10, Loss_val: 0.474884
Epoch: 1, Step: 11, Loss: 0.406503
Epoch: 1, Step: 12, Loss: 0.409969
Epoch: 1, Step: 13, Loss: 0.561448
Epoch: 1, Step: 14, Loss: 0.414018
Epoch: 1, Step: 15, Loss: 0.326028
Epoch: 1, Step: 16, Loss: 0.389219
Epoch: 1, Step: 17, Loss: 0.424883
Epoch: 1, Step: 18, Loss: 0.234388
Epoch: 1, Step: 19, Loss: 0.313719
Epoch: 

Epoch: 1, Step: 201, Loss: 0.295942
Epoch: 1, Step: 202, Loss: 0.342467
Epoch: 1, Step: 203, Loss: 0.350426
Epoch: 1, Step: 204, Loss: 0.381146
Epoch: 1, Step: 205, Loss: 0.34594
Epoch: 1, Step: 206, Loss: 0.287952
Epoch: 1, Step: 207, Loss: 0.343473
Epoch: 1, Step: 208, Loss: 0.301471
Epoch: 1, Step: 209, Loss: 0.219463
Epoch: 1, Step: 210, Loss: 0.306897
Epoch: 1, Step: 210, Loss_val: 0.313835
Epoch: 1, Step: 211, Loss: 0.341154
Epoch: 1, Step: 212, Loss: 0.351
Epoch: 1, Step: 213, Loss: 0.278047
Epoch: 1, Step: 214, Loss: 0.248268
Epoch: 1, Step: 215, Loss: 0.310576
Epoch: 1, Step: 216, Loss: 0.247202
Epoch: 1, Step: 217, Loss: 0.314987
Epoch: 1, Step: 218, Loss: 0.367928
Epoch: 1, Step: 219, Loss: 0.311973
Epoch: 1, Step: 220, Loss: 0.279451
Epoch: 1, Step: 220, Loss_val: 0.372136
Epoch: 1, Step: 221, Loss: 0.422678
Epoch: 1, Step: 222, Loss: 0.365543
Epoch: 1, Step: 223, Loss: 0.37326
Epoch: 1, Step: 224, Loss: 0.276966
Epoch: 1, Step: 225, Loss: 0.291017
Epoch: 1, Step: 226, Loss

Epoch: 2, Step: 3, Loss: 0.310749
Epoch: 2, Step: 4, Loss: 0.307722
Epoch: 2, Step: 5, Loss: 0.381969
Epoch: 2, Step: 6, Loss: 0.505768
Epoch: 2, Step: 7, Loss: 0.337573
Epoch: 2, Step: 8, Loss: 0.214685
Epoch: 2, Step: 9, Loss: 0.176155
Epoch: 2, Step: 10, Loss: 0.361144
Epoch: 2, Step: 10, Loss_val: 0.344441
Epoch: 2, Step: 11, Loss: 0.320231
Epoch: 2, Step: 12, Loss: 0.276946
Epoch: 2, Step: 13, Loss: 0.322989
Epoch: 2, Step: 14, Loss: 0.220382
Epoch: 2, Step: 15, Loss: 0.22732
Epoch: 2, Step: 16, Loss: 0.297662
Epoch: 2, Step: 17, Loss: 0.27389
Epoch: 2, Step: 18, Loss: 0.234348
Epoch: 2, Step: 19, Loss: 0.257066
Epoch: 2, Step: 20, Loss: 0.168437
Epoch: 2, Step: 20, Loss_val: 0.328862
Epoch: 2, Step: 21, Loss: 0.387018
Epoch: 2, Step: 22, Loss: 0.382382
Epoch: 2, Step: 23, Loss: 0.318119
Epoch: 2, Step: 24, Loss: 0.346252
Epoch: 2, Step: 25, Loss: 0.302317
Epoch: 2, Step: 26, Loss: 0.322985
Epoch: 2, Step: 27, Loss: 0.218761
Epoch: 2, Step: 28, Loss: 0.300444
Epoch: 2, Step: 29, L

...

Epoch: 27, Step: 351, Loss: 0.072799
Epoch: 27, Step: 352, Loss: 0.0604475
Epoch: 27, Step: 353, Loss: 0.0574748
Epoch: 27, Step: 354, Loss: 0.098036
Epoch: 27, Step: 355, Loss: 0.0495588
Epoch: 27, Step: 356, Loss: 0.0308473
Epoch: 27, Step: 357, Loss: 0.107868
Epoch: 27, Step: 358, Loss: 0.0509999
Epoch: 27, Step: 359, Loss: 0.0687962
Epoch: 27, Step: 360, Loss: 0.106544
Epoch: 27, Step: 360, Loss_val: 0.083111
Epoch: 27, Step: 361, Loss: 0.107093
Epoch: 27, Step: 362, Loss: 0.0201357
Epoch: 27, Step: 363, Loss: 0.114208
Epoch: 27, Step: 364, Loss: 0.125455
Epoch: 27, Step: 365, Loss: 0.0676389
Epoch: 27, Step: 366, Loss: 0.100052
Epoch: 27, Step: 367, Loss: 0.0521701
Epoch: 27, Step: 368, Loss: 0.0545705
Epoch: 27, Step: 369, Loss: 0.02658
Epoch: 27, Step: 370, Loss: 0.0371877
Epoch: 27, Step: 370, Loss_val: 0.11103
Epoch: 27, Step: 371, Loss: 0.0610175
Epoch: 27, Step: 372, Loss: 0.0722837
Epoch: 27, Step: 373, Loss: 0.0755243
Epoch: 27, Step: 374, Loss: 0.0176931
Epoch: 27, Step: 

Epoch: 28, Step: 146, Loss: 0.019469
Epoch: 28, Step: 147, Loss: 0.0520795
Epoch: 28, Step: 148, Loss: 0.115642
Epoch: 28, Step: 149, Loss: 0.0765279
Epoch: 28, Step: 150, Loss: 0.119505
Epoch: 28, Step: 150, Loss_val: 0.129417
Epoch: 28, Step: 151, Loss: 0.0702349
Epoch: 28, Step: 152, Loss: 0.0638091
Epoch: 28, Step: 153, Loss: 0.068184
Epoch: 28, Step: 154, Loss: 0.0769923
Epoch: 28, Step: 155, Loss: 0.0602215
Epoch: 28, Step: 156, Loss: 0.0480502
Epoch: 28, Step: 157, Loss: 0.0797172
Epoch: 28, Step: 158, Loss: 0.0399628
Epoch: 28, Step: 159, Loss: 0.0479504
Epoch: 28, Step: 160, Loss: 0.0533844
Epoch: 28, Step: 160, Loss_val: 0.221393
Epoch: 28, Step: 161, Loss: 0.0624114
Epoch: 28, Step: 162, Loss: 0.101554
Epoch: 28, Step: 163, Loss: 0.0502449
Epoch: 28, Step: 164, Loss: 0.0511645
Epoch: 28, Step: 165, Loss: 0.0951016
Epoch: 28, Step: 166, Loss: 0.0428289
Epoch: 28, Step: 167, Loss: 0.0391913
Epoch: 28, Step: 168, Loss: 0.0743204
Epoch: 28, Step: 169, Loss: 0.0264924
Epoch: 28, 

Epoch: 28, Step: 341, Loss: 0.0987433
Epoch: 28, Step: 342, Loss: 0.128312
Epoch: 28, Step: 343, Loss: 0.0640587
Epoch: 28, Step: 344, Loss: 0.092571
Epoch: 28, Step: 345, Loss: 0.10317
Epoch: 28, Step: 346, Loss: 0.119781
Epoch: 28, Step: 347, Loss: 0.0835837
Epoch: 28, Step: 348, Loss: 0.0949961
Epoch: 28, Step: 349, Loss: 0.0637244
Epoch: 28, Step: 350, Loss: 0.103829
Epoch: 28, Step: 350, Loss_val: 0.0998621
Epoch: 28, Step: 351, Loss: 0.122503
Epoch: 28, Step: 352, Loss: 0.147982
Epoch: 28, Step: 353, Loss: 0.0821837
Epoch: 28, Step: 354, Loss: 0.0524868
Epoch: 28, Step: 355, Loss: 0.176462
Epoch: 28, Step: 356, Loss: 0.117113
Epoch: 28, Step: 357, Loss: 0.0953069
Epoch: 28, Step: 358, Loss: 0.085118
Epoch: 28, Step: 359, Loss: 0.244802
Epoch: 28, Step: 360, Loss: 0.167127
Epoch: 28, Step: 360, Loss_val: 0.185352
Epoch: 28, Step: 361, Loss: 0.115295
Epoch: 28, Step: 362, Loss: 0.0731051
Epoch: 28, Step: 363, Loss: 0.062221
Epoch: 28, Step: 364, Loss: 0.079996
Epoch: 28, Step: 365,

Epoch: 29, Step: 137, Loss: 0.0528771
Epoch: 29, Step: 138, Loss: 0.109948
Epoch: 29, Step: 139, Loss: 0.0539445
Epoch: 29, Step: 140, Loss: 0.0361576
Epoch: 29, Step: 140, Loss_val: 0.0929303
Epoch: 29, Step: 141, Loss: 0.0427315
Epoch: 29, Step: 142, Loss: 0.0655375
Epoch: 29, Step: 143, Loss: 0.0585128
Epoch: 29, Step: 144, Loss: 0.133712
Epoch: 29, Step: 145, Loss: 0.11203
Epoch: 29, Step: 146, Loss: 0.0781862
Epoch: 29, Step: 147, Loss: 0.0586261
Epoch: 29, Step: 148, Loss: 0.0782556
Epoch: 29, Step: 149, Loss: 0.0312071
Epoch: 29, Step: 150, Loss: 0.0375948
Epoch: 29, Step: 150, Loss_val: 0.165401
Epoch: 29, Step: 151, Loss: 0.0560462
Epoch: 29, Step: 152, Loss: 0.0612896
Epoch: 29, Step: 153, Loss: 0.0722894
Epoch: 29, Step: 154, Loss: 0.0531877
Epoch: 29, Step: 155, Loss: 0.0795146
Epoch: 29, Step: 156, Loss: 0.0561676
Epoch: 29, Step: 157, Loss: 0.0421572
Epoch: 29, Step: 158, Loss: 0.0579602
Epoch: 29, Step: 159, Loss: 0.0530084
Epoch: 29, Step: 160, Loss: 0.133911
Epoch: 29,

Epoch: 29, Step: 331, Loss: 0.0800825
Epoch: 29, Step: 332, Loss: 0.140751
Epoch: 29, Step: 333, Loss: 0.0655864
Epoch: 29, Step: 334, Loss: 0.0756717
Epoch: 29, Step: 335, Loss: 0.071462
Epoch: 29, Step: 336, Loss: 0.0790534
Epoch: 29, Step: 337, Loss: 0.15947
Epoch: 29, Step: 338, Loss: 0.079716
Epoch: 29, Step: 339, Loss: 0.113746
Epoch: 29, Step: 340, Loss: 0.162536
Epoch: 29, Step: 340, Loss_val: 0.143445
Epoch: 29, Step: 341, Loss: 0.0580629
Epoch: 29, Step: 342, Loss: 0.0890255
Epoch: 29, Step: 343, Loss: 0.0600417
Epoch: 29, Step: 344, Loss: 0.0455634
Epoch: 29, Step: 345, Loss: 0.0475236
Epoch: 29, Step: 346, Loss: 0.139974
Epoch: 29, Step: 347, Loss: 0.0981879
Epoch: 29, Step: 348, Loss: 0.106029
Epoch: 29, Step: 349, Loss: 0.0802751
Epoch: 29, Step: 350, Loss: 0.0530816
Epoch: 29, Step: 350, Loss_val: 0.133868
Epoch: 29, Step: 351, Loss: 0.0284075
Epoch: 29, Step: 352, Loss: 0.132878
Epoch: 29, Step: 353, Loss: 0.087497
Epoch: 29, Step: 354, Loss: 0.0585775
Epoch: 29, Step: 